In [1]:
import os
import json
import glob
import numpy as np
from tqdm import tqdm
from PIL import Image

class YoloToLabelmeConverter:
    def __init__(self, dataset_dir, class_names):
        self.dataset_dir = os.path.abspath(dataset_dir)
        self.class_names = class_names
        self.txt_files = []
        self.img_files = []
    
    def find_files(self):
        print(f"Searching for files in: {self.dataset_dir}")
        
        if not os.path.exists(self.dataset_dir):
            print(f"Error: Directory {self.dataset_dir} does not exist")
            return
            
        txt_pattern = os.path.join(self.dataset_dir, "**/*.txt")
        print(f"Using pattern: {txt_pattern}")
        
        all_files = glob.glob(txt_pattern, recursive=True)
        print(f"Total files found: {len(all_files)}")
        
        self.txt_files = []
        self.img_files = []
        img_extensions = ['.jpg', '.jpeg', '.png']
        
        for txt_file in all_files:
            if os.path.basename(txt_file) == "classes.txt":
                continue
                
            found_img = False
            for ext in img_extensions:
                img_file = os.path.splitext(txt_file)[0] + ext
                if os.path.exists(img_file):
                    self.txt_files.append(txt_file)
                    self.img_files.append(img_file)
                    found_img = True
                    break
            
            if not found_img:
                print(f"No image found for: {txt_file}")
        
        print(f"Found {len(self.txt_files)} annotation files with corresponding images")
        
        if len(self.txt_files) == 0:
            print("Listing all .txt files found:")
            for txt in all_files:
                print(f"- {txt}")
    
    def is_segmentation(self, parts):
        return len(parts) > 5 and (len(parts) - 1) % 2 == 0
    
    def convert_yolo_to_labelme(self, txt_file, img_file):
        try:
            img = Image.open(img_file)
            img_width, img_height = img.size
            
            with open(txt_file, 'r') as f:
                lines = f.read().strip().split('\n')
            
            shapes = []
            for line in lines:
                if not line.strip():
                    continue
                    
                parts = line.strip().split()
                if len(parts) < 5:
                    print(f"Warning: Invalid format in {txt_file}: {line}")
                    continue
                
                class_id = int(parts[0])
                if class_id >= len(self.class_names):
                    print(f"Warning: Class ID {class_id} is out of range in {txt_file}")
                    continue
                
                if self.is_segmentation(parts):
                    points = []
                    for i in range(1, len(parts), 2):
                        x = float(parts[i]) * img_width
                        y = float(parts[i+1]) * img_height
                        points.append([x, y])
                    
                    shape = {
                        "label": self.class_names[class_id],
                        "points": points,
                        "group_id": None,
                        "shape_type": "polygon",
                        "flags": {}
                    }
                else:
                    x_center = float(parts[1])
                    y_center = float(parts[2])
                    width = float(parts[3])
                    height = float(parts[4])
                    
                    x_min = (x_center - width/2) * img_width
                    y_min = (y_center - height/2) * img_height
                    x_max = (x_center + width/2) * img_width
                    y_max = (y_center + height/2) * img_height
                    
                    shape = {
                        "label": self.class_names[class_id],
                        "points": [[x_min, y_min], [x_max, y_max]],
                        "group_id": None,
                        "shape_type": "rectangle",
                        "flags": {}
                    }
                
                shapes.append(shape)
            
            labelme_data = {
                "version": "4.5.7",
                "flags": {},
                "shapes": shapes,
                "imagePath": os.path.basename(img_file),
                "imageData": None,
                "imageHeight": img_height,
                "imageWidth": img_width
            }
            
            json_path = os.path.splitext(img_file)[0] + ".json"
            with open(json_path, 'w') as f:
                json.dump(labelme_data, f, indent=2)
            
            return True
            
        except Exception as e:
            print(f"Error converting {txt_file}: {str(e)}")
            return False
    
    def convert_all(self):
        if not self.txt_files:
            print("No files to convert.")
            return
            
        success_count = 0
        
        for txt_file, img_file in tqdm(zip(self.txt_files, self.img_files), total=len(self.txt_files), desc="Converting annotations"):
            if self.convert_yolo_to_labelme(txt_file, img_file):
                success_count += 1
        
        print(f"Successfully converted {success_count}/{len(self.txt_files)} annotations")

dataset_dir = r'D:\bai_toan\weapon\13_4\tong\0380'
class_names = ['person', 'gun', 'knife', 'stick']

converter = YoloToLabelmeConverter(dataset_dir, class_names)
converter.find_files()
converter.convert_all()

Searching for files in: D:\bai_toan\weapon\13_4\tong\0380
Using pattern: D:\bai_toan\weapon\13_4\tong\0380\**/*.txt
Total files found: 332
Found 332 annotation files with corresponding images


Converting annotations: 100%|██████████| 332/332 [00:19<00:00, 16.82it/s]

Successfully converted 332/332 annotations


In [1]:
import os
import glob

def remove_files(directory_path, extension=None):
    pattern = os.path.join(directory_path, f"*.{extension}" if extension else "*")
    files = glob.glob(pattern)
    count = 0
    for file_path in files:
        if os.path.isfile(file_path):
            os.remove(file_path)
            count += 1
    return f"Removed {count} files from {directory_path}"

result = remove_files(r"/media/tat/Learn1/bai_toan/weapon/4_9_min/DJI_0326", "json")

In [ ]:
import os
import glob

def clean_unpaired_images(directory_path):
    print(f"Đang xử lý thư mục: {directory_path}")

    image_files = glob.glob(os.path.join(directory_path, "*.jpg"))
    print(f"Tổng số file ảnh (.jpg): {len(image_files)}")
    
    unpaired_count = 0
    deleted_count = 0

    for image_path in image_files:
        # Tạo tên file JSON tương ứng
        json_path = os.path.splitext(image_path)[0] + ".json"
        if not os.path.exists(json_path):
            unpaired_count += 1
            print(f"File ảnh không có cặp: {os.path.basename(image_path)}")
            
            try:
                os.remove(image_path)
                deleted_count += 1
                print(f"Đã xóa: {os.path.basename(image_path)}")
            except Exception as e:
                print(f"Lỗi khi xóa file {os.path.basename(image_path)}: {str(e)}")
    
    print(f"\nKết quả:")
    print(f"- Tổng số file ảnh (.jpg): {len(image_files)}")
    print(f"- Số file ảnh không có file JSON tương ứng: {unpaired_count}")
    print(f"- Số file ảnh đã xóa thành công: {deleted_count}")

if __name__ == "__main__":
    dataset_dir = r"D:\bai_toan\Traffic\Accident and Non-accident label Image Dataset.v14-hai-s-augment-attempt.yolov11\dataset"

    if not os.path.isdir(dataset_dir):
        print(f"Thư mục không tồn tại: {dataset_dir}")
    else:
        clean_unpaired_images(dataset_dir)
        print("Hoàn thành xử lý!")

In [16]:
import os
import json
import glob
import shutil
from pathlib import Path
from tqdm import tqdm

class LabelmeClassRemover:
    def __init__(self, input_dir, output_dir, classes_to_remove):
        self.input_dir = input_dir
        self.output_dir = output_dir
        self.classes_to_remove = classes_to_remove
        self.label_counts = {}
        Path(output_dir).mkdir(parents=True, exist_ok=True)
        
    def process_files(self):
        json_files = glob.glob(os.path.join(self.input_dir, "*.json"))
        
        for json_file in tqdm(json_files, desc="Processing files"):
            try:
                base_filename = os.path.basename(json_file)
                image_filename = os.path.splitext(base_filename)[0]
                
                for ext in ['.jpg', '.jpeg', '.png']:
                    image_path = os.path.join(self.input_dir, f"{image_filename}{ext}")
                    if os.path.exists(image_path):
                        break
                else:
                    continue
                
                with open(json_file, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                
                if "shapes" in data:
                    original_shapes_count = len(data["shapes"])
                    data["shapes"] = [shape for shape in data["shapes"] if shape.get("label") not in self.classes_to_remove]
                    
                    if len(data["shapes"]) > 0:
                        for shape in data["shapes"]:
                            label = shape.get("label", "unknown")
                            self.label_counts[label] = self.label_counts.get(label, 0) + 1
                        
                        output_json_path = os.path.join(self.output_dir, base_filename)
                        with open(output_json_path, 'w', encoding='utf-8') as f:
                            json.dump(data, f, indent=2, ensure_ascii=False)
                        
                        shutil.copy(image_path, os.path.join(self.output_dir, os.path.basename(image_path)))
            except Exception as e:
                print(f"Error processing {json_file}: {str(e)}")
    
    def print_statistics(self):
        print("\nRemaining Label Statistics:")
        print("=" * 40)
        
        sorted_labels = sorted(self.label_counts.items(), key=lambda x: x[1], reverse=True)
        
        for label, count in sorted_labels:
            print(f"{label}: {count} instances")
        
        print("=" * 40)
        print(f"Total unique labels: {len(self.label_counts)}")
        print(f"Total annotations: {sum(self.label_counts.values())}")

def main():
    input_dir = r"D:\bai_toan\car\new\Data_drone_traffic\lion.v7-yolov9.yolov11\dataset"
    output_dir = r"D:\bai_toan\car\new\Data_drone_traffic\lion.v7-yolov9.yolov11\output"
    classes_to_remove = ["person", "trailer", "bicycle"]
    
    remover = LabelmeClassRemover(input_dir, output_dir, classes_to_remove)
    remover.process_files()
    remover.print_statistics()

if __name__ == "__main__":
    main()

Processing files: 100%|██████████| 6085/6085 [01:02<00:00, 97.97it/s] 


Remaining Label Statistics:
car: 43621 instances
van: 2126 instances
motorbike: 386 instances
truck: 339 instances
mini_truck: 332 instances
bus: 124 instances
Total unique labels: 6
Total annotations: 46928


# gộp nhãn

In [17]:
import os
import json
import glob

def convert_labels_in_json(dataset_path):
    car_labels = ['bicycle', 'bus', 'car', 'mini_truck', 'trailer', 'truck', 'van']
    moto_labels = ['motorbike']
    
    json_files = glob.glob(os.path.join(dataset_path, "*.json"))
    
    print(f"Tìm thấy {len(json_files)} file JSON trong thư mục {dataset_path}")
    
    modified_count = 0
    
    for json_file in json_files:
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            if 'shapes' not in data:
                print(f"Bỏ qua file {json_file}: Không phải định dạng labelme")
                continue
                
            modified = False
            
            for shape in data['shapes']:
                if 'label' in shape:
                    original_label = shape['label'].lower()
                    
                    if original_label in car_labels:
                        shape['label'] = 'car'
                        modified = True
                    elif original_label in moto_labels:
                        shape['label'] = 'motor'
                        modified = True
                    elif original_label == 'person':
                        shape.pop('label')
                        modified = True
            
            if modified:
                with open(json_file, 'w', encoding='utf-8') as f:
                    json.dump(data, f, indent=2, ensure_ascii=False)
                modified_count += 1
                
        except Exception as e:
            print(f"Lỗi khi xử lý file {json_file}: {str(e)}")
    
    print(f"Đã chuyển đổi nhãn thành công trong {modified_count}/{len(json_files)} file JSON")

dataset_path = r"D:\bai_toan\car\new\Data_drone_traffic\lion.v7-yolov9.yolov11\output"
convert_labels_in_json(dataset_path)

Tìm thấy 5995 file JSON trong thư mục D:\bai_toan\car\new\Data_drone_traffic\lion.v7-yolov9.yolov11\output
Đã chuyển đổi nhãn thành công trong 5995/5995 file JSON


# seg -> box

In [ ]:
import os
import json
import glob
import numpy as np

def convert_segmentation_to_bbox(dataset_path):
    # Tìm tất cả các file JSON trong thư mục
    json_files = glob.glob(os.path.join(dataset_path, "*.json"))
    
    print(f"Tìm thấy {len(json_files)} file JSON trong thư mục {dataset_path}")
    
    modified_count = 0
    
    # Xử lý từng file JSON
    for json_file in json_files:
        try:
            # Đọc file JSON
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            # Kiểm tra xem file có trường shapes không (định dạng labelme)
            if 'shapes' not in data:
                print(f"Bỏ qua file {json_file}: Không phải định dạng labelme")
                continue
            
            modified = False
            
            # Duyệt qua các đối tượng trong file
            for shape in data['shapes']:
                # Chỉ xử lý các đối tượng là polygon hoặc segmentation
                if shape['shape_type'] in ['polygon', 'linestrip']:
                    # Chuyển đổi thành bounding box
                    points = np.array(shape['points'])
                    
                    # Tính tọa độ min và max để tạo bounding box
                    x_min = np.min(points[:, 0])
                    y_min = np.min(points[:, 1])
                    x_max = np.max(points[:, 0])
                    y_max = np.max(points[:, 1])
                    
                    # Tạo bounding box với 2 điểm (top-left, bottom-right) để tránh lỗi
                    shape['points'] = [[x_min, y_min], [x_max, y_max]]
                    shape['shape_type'] = 'rectangle'
                    
                    modified = True
            
            # Nếu có sự thay đổi, lưu lại file
            if modified:
                with open(json_file, 'w', encoding='utf-8') as f:
                    json.dump(data, f, indent=2, ensure_ascii=False)
                modified_count += 1
                print(f"Đã chuyển đổi file: {os.path.basename(json_file)}")
                
        except Exception as e:
            print(f"Lỗi khi xử lý file {json_file}: {str(e)}")
    
    print(f"Đã chuyển đổi thành công {modified_count}/{len(json_files)} file JSON từ segmentation sang bounding box")

# Thực thi hàm với đường dẫn đến thư mục dataset
dataset_path = r"D:\bai_toan\car\new\Data_drone_traffic\lion.v7-yolov9.yolov11\output"
convert_segmentation_to_bbox(dataset_path)

# xóa

In [12]:
import os
import glob

def remove_files(directory_path, extension=None):
    pattern = os.path.join(directory_path, f"*.{extension}" if extension else "*")
    files = glob.glob(pattern)
    count = 0
    for file_path in files:
        if os.path.isfile(file_path):
            os.remove(file_path)
            count += 1
    return f"Removed {count} files from {directory_path}"

result = remove_files("D:/bai_toan/car/new/Data_drone_traffic/Proj.v2i.yolov11/dataset", "txt")

# json -> txt

In [19]:
import os
import json
import shutil
import random
import numpy as np
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

class LabelmeToYoloConverter:
    def __init__(self, source_dir, class_mapping=None):
        self.source_dir = source_dir
        self.dataset_dir = os.path.join(source_dir, 'dataset')
        self.train_dir = os.path.join(self.dataset_dir, 'train')
        self.val_dir = os.path.join(self.dataset_dir, 'val')
        
        self.images = []
        self.annotations = []
        
        if class_mapping is None:
            self.class_mapping = {'car': 0, 'motor': 1}
        else:
            self.class_mapping = class_mapping
            
    def setup_directories(self):
        os.makedirs(self.dataset_dir, exist_ok=True)
        os.makedirs(self.train_dir, exist_ok=True)
        os.makedirs(self.val_dir, exist_ok=True)
        
    def find_files(self):
        image_extensions = ['*.jpg', '*.jpeg', '*.png']
        self.images = []
        
        for ext in image_extensions:
            self.images.extend(glob(os.path.join(self.source_dir, ext)))
            
        self.annotations = glob(os.path.join(self.source_dir, '*.json'))
        
        print(f"Found {len(self.images)} images and {len(self.annotations)} json annotations")
        
    def get_class_mapping_from_annotations(self):
        classes = set()
        
        for json_path in tqdm(self.annotations, desc="Scanning classes"):
            try:
                with open(json_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    
                for shape in data.get('shapes', []):
                    label = shape.get('label', '').strip().lower()
                    if label and label not in classes:
                        classes.add(label)
            except Exception as e:
                print(f"Error reading {json_path}: {e}")
                
        class_mapping = {cls_name: idx for idx, cls_name in enumerate(sorted(classes))}
        print(f"Detected classes: {class_mapping}")
        return class_mapping
    
    def convert_annotation(self, json_path, image_width, image_height):
        yolo_annotations = []
        
        try:
            with open(json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                
            for shape in data.get('shapes', []):
                label = shape.get('label', '').strip().lower()
                if label not in self.class_mapping:
                    continue
                    
                class_id = self.class_mapping[label]
                shape_type = shape.get('shape_type', '')
                points = shape.get('points', [])
                
                if shape_type == 'rectangle' and len(points) == 2:
                    x1, y1 = points[0]
                    x2, y2 = points[1]
                    
                    x_min, x_max = min(x1, x2), max(x1, x2)
                    y_min, y_max = min(y1, y2), max(y1, y2)
                    
                    x_center = (x_min + x_max) / 2 / image_width
                    y_center = (y_min + y_max) / 2 / image_height
                    width = (x_max - x_min) / image_width
                    height = (y_max - y_min) / image_height
                    
                    yolo_annotations.append(
                        f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
                    )
                elif len(points) >= 3:
                    xs = [p[0] for p in points]
                    ys = [p[1] for p in points]

                    x_min, x_max = min(xs), max(xs)
                    y_min, y_max = min(ys), max(ys)

                    x_center = (x_min + x_max) / 2 / image_width
                    y_center = (y_min + y_max) / 2 / image_height
                    width = (x_max - x_min) / image_width
                    height = (y_max - y_min) / image_height

                    yolo_annotations.append(
                        f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
                    )
                
        except Exception as e:
            print(f"Error converting {json_path}: {e}")
            
        return yolo_annotations
    
    def process_dataset(self, test_size=0.2, random_state=42):
        self.setup_directories()
        self.find_files()
        
        if not self.class_mapping:
            self.class_mapping = self.get_class_mapping_from_annotations()

        data_pairs = []
        for img_path in tqdm(self.images, desc="Processing images"):
            base_name = os.path.splitext(os.path.basename(img_path))[0]
            json_path = os.path.join(self.source_dir, f"{base_name}.json")
            
            if not os.path.exists(json_path):
                print(f"Warning: No JSON file found for {img_path}")
                continue
                
            data_pairs.append((img_path, json_path))

        train_pairs, val_pairs = train_test_split(
            data_pairs, test_size=test_size, random_state=random_state
        )

        self._process_dataset_split(train_pairs, self.train_dir)
        self._process_dataset_split(val_pairs, self.val_dir)
        
        print(f"Dataset conversion complete. Train: {len(train_pairs)}, Validation: {len(val_pairs)}")
        
    def _process_dataset_split(self, data_pairs, output_dir):
        from PIL import Image
        
        for img_path, json_path in tqdm(data_pairs, desc=f"Processing {os.path.basename(output_dir)}"):
            try:
                with Image.open(img_path) as img:
                    img_width, img_height = img.size

                yolo_annotations = self.convert_annotation(json_path, img_width, img_height)
                
                if not yolo_annotations:
                    continue

                dest_img_path = os.path.join(output_dir, os.path.basename(img_path))
                shutil.copy2(img_path, dest_img_path)

                base_name = os.path.splitext(os.path.basename(img_path))[0]
                txt_path = os.path.join(output_dir, f"{base_name}.txt")
                
                with open(txt_path, 'w', encoding='utf-8') as f:
                    f.write('\n'.join(yolo_annotations))
                    
            except Exception as e:
                print(f"Error processing {img_path}: {e}")

source_directory = r"D:\bai_toan\car\new\Data_drone_traffic\lion.v7-yolov9.yolov11\output"

class_mapping = {
    'car': 0,
    'motor': 1  # Changed from 'Motor' to 'motor' (lowercase)
}

converter = LabelmeToYoloConverter(source_directory, class_mapping)
converter.process_dataset(test_size=0.2, random_state=42)

print("\nDataset conversion summary:")
print(f"Source directory: {source_directory}")
print(f"Output directory: {os.path.join(source_directory, 'dataset')}")
print(f"Class mapping: {converter.class_mapping}")

Found 5995 images and 5995 json annotations


Processing val: 100%|██████████| 1199/1199 [00:31<00:00, 38.01it/s]

Dataset conversion complete. Train: 4796, Validation: 1199

Dataset conversion summary:
Source directory: D:\bai_toan\car\new\Data_drone_traffic\lion.v7-yolov9.yolov11\output
Output directory: D:\bai_toan\car\new\Data_drone_traffic\lion.v7-yolov9.yolov11\output\dataset
Class mapping: {'car': 0, 'motor': 1}


In [3]:
import os
import glob
from concurrent.futures import ThreadPoolExecutor
import time

class FileRemover:
    def __init__(self):
        self.dataset_tong_dir = r"D:\bai_toan\weapon\dataset_tong"
        self.target_dir = r"D:\bai_toan\weapon\3_26\tong\obb\dataset"
        self.train_dir = os.path.join(self.target_dir, "train")
        self.val_dir = os.path.join(self.target_dir, "val")
        self.image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff']
        self.removed_count = 0
        self.total_files = 0
        self.start_time = time.time()

    def get_image_names(self):
        image_names = set()
        for ext in self.image_extensions:
            pattern = os.path.join(self.dataset_tong_dir, f"*{ext}")
            files = glob.glob(pattern)
            for file_path in files:
                basename = os.path.basename(file_path)
                name_without_ext = os.path.splitext(basename)[0]
                image_names.add(name_without_ext)
        return image_names

    def remove_files_from_dir(self, directory, image_basenames, batch_size=1000):
        batches = [list(image_basenames)[i:i+batch_size] for i in range(0, len(image_basenames), batch_size)]
        
        for batch_idx, batch in enumerate(batches):
            with ThreadPoolExecutor(max_workers=8) as executor:
                futures = []
                for basename in batch:
                    for ext in self.image_extensions:
                        img_path = os.path.join(directory, f"{basename}{ext}")
                        if os.path.exists(img_path):
                            futures.append(executor.submit(os.remove, img_path))
                            
                    txt_path = os.path.join(directory, f"{basename}.txt")
                    if os.path.exists(txt_path):
                        futures.append(executor.submit(os.remove, txt_path))
                
                completed = 0
                for future in futures:
                    try:
                        future.result()
                        completed += 1
                    except Exception:
                        pass
            
            self.removed_count += completed
            elapsed = time.time() - self.start_time
            remaining_batches = len(batches) - (batch_idx + 1)
            avg_time_per_batch = elapsed / (batch_idx + 1)
            estimated_remaining = avg_time_per_batch * remaining_batches
            
            print(f"Batch {batch_idx+1}/{len(batches)} completed. Total removed: {self.removed_count}")
            print(f"Elapsed: {elapsed:.1f}s, Est. remaining: {estimated_remaining:.1f}s")
            
        return self.removed_count

    def run(self):
        print("Đang lấy danh sách tên ảnh từ dataset_tong...")
        image_basenames = self.get_image_names()
        print(f"Tìm thấy {len(image_basenames)} tên ảnh trong dataset_tong")
        
        print("Đang xóa files từ thư mục train...")
        train_removed = self.remove_files_from_dir(self.train_dir, image_basenames)
        
        print("Đang xóa files từ thư mục val...")
        val_removed = self.remove_files_from_dir(self.val_dir, image_basenames)
        
        print(f"Hoàn tất! Đã xóa tổng cộng {self.removed_count} files")
        print(f"Thời gian thực hiện: {time.time() - self.start_time:.1f} giây")

if __name__ == "__main__":
    remover = FileRemover()
    remover.run()

Đang lấy danh sách tên ảnh từ dataset_tong...
Tìm thấy 22494 tên ảnh trong dataset_tong
Đang xóa files từ thư mục train...
Batch 1/23 completed. Total removed: 1086
Elapsed: 1.7s, Est. remaining: 38.4s
Batch 2/23 completed. Total removed: 2252
Elapsed: 2.4s, Est. remaining: 25.2s
Batch 3/23 completed. Total removed: 3342
Elapsed: 3.0s, Est. remaining: 19.8s
Batch 4/23 completed. Total removed: 4490
Elapsed: 3.7s, Est. remaining: 17.5s
Batch 5/23 completed. Total removed: 5614
Elapsed: 4.2s, Est. remaining: 15.3s
Batch 6/23 completed. Total removed: 6694
Elapsed: 4.9s, Est. remaining: 13.9s
Batch 7/23 completed. Total removed: 7848
Elapsed: 5.5s, Est. remaining: 12.5s
Batch 8/23 completed. Total removed: 8978
Elapsed: 6.0s, Est. remaining: 11.3s
Batch 9/23 completed. Total removed: 10090
Elapsed: 6.7s, Est. remaining: 10.4s
Batch 10/23 completed. Total removed: 11174
Elapsed: 7.2s, Est. remaining: 9.4s
Batch 11/23 completed. Total removed: 12356
Elapsed: 7.9s, Est. remaining: 8.6s
Batch